# Python API Example - Access Slots Import and Storage in Dataframe

This guide is designed to provide an example of how to access the Spark API:
- The path to your client credentials is the only input needed to run this script (just before Section 2)
- This script has been designed to display the raw outputs of requests from the API
- This script can be copied and pasted by customers for quick use of the API

__N.B. This guide is just for Access terminal data. If you're looking for other API data products (such as contract prices, Freight routes or Netbacks), please refer to their according code example files.__ 

### Have any questions?

If you have any questions regarding our API, or need help accessing specific datasets, please contact us at:

__data@sparkcommodities.com__

or refer to our API website for more information about this endpoint: https://www.sparkcommodities.com/api/request/access.html

## 1. Importing Data

Here we define the functions that allow us to retrieve the valid credentials to access the Spark API.

This section can remain unchanged for most Spark API users.

In [4]:
# import libraries for importing data
import json
import os
import sys
import pandas as pd
from base64 import b64encode
from urllib.parse import urljoin
from pprint import pprint
import requests
from io import StringIO
import time

try:
    from urllib import request, parse
    from urllib.error import HTTPError
except ImportError:
    raise RuntimeError("Python 3 required")


In [5]:
# defining functions 
API_BASE_URL = "https://api.sparkcommodities.com"


def retrieve_credentials(file_path=None):
    """
    Find credentials either by reading the client_credentials file or reading
    environment variables
    """
    if file_path is None:
        client_id = os.getenv("SPARK_CLIENT_ID")
        client_secret = os.getenv("SPARK_CLIENT_SECRET")
        if not client_id or not client_secret:
            raise RuntimeError(
                "SPARK_CLIENT_ID and SPARK_CLIENT_SECRET environment vars required"
            )
    else:
        # Parse the file
        if not os.path.isfile(file_path):
            raise RuntimeError("The file {} doesn't exist".format(file_path))

        with open(file_path) as fp:
            lines = [l.replace("\n", "") for l in fp.readlines()]

        if lines[0] in ("clientId,clientSecret", "client_id,client_secret"):
            client_id, client_secret = lines[1].split(",")
        else:
            print("First line read: '{}'".format(lines[0]))
            raise RuntimeError(
                "The specified file {} doesn't look like to be a Spark API client "
                "credentials file".format(file_path)
            )

    print(">>>> Found credentials!")
    print(
        ">>>> Client_id={}****, client_secret={}****".format(
            client_id[:5], client_secret[:5]
        )
    )

    return client_id, client_secret

def do_api_post_query(uri, body, headers):
    """
    OAuth2 authentication requires a POST request with client credentials before accessing the API. 
    This POST request will return an Access Token which will be used for the API GET request.
    """
    url = urljoin(API_BASE_URL, uri)

    data = json.dumps(body).encode("utf-8")

    # HTTP POST request
    req = request.Request(url, data=data, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 201, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content

def get_access_token(client_id, client_secret):
    """
    Get a new access_token. Access tokens are the thing that applications use to make
    API requests. Access tokens must be kept confidential in storage.

    # Procedure:

    Do a POST query with `grantType` and `scopes` in the body. A basic authorization
    HTTP header is required. The "Basic" HTTP authentication scheme is defined in
    RFC 7617, which transmits credentials as `clientId:clientSecret` pairs, encoded
    using base64.
    """

    # Note: for the sake of this example, we choose to use the Python urllib from the
    # standard lib. One should consider using https://requests.readthedocs.io/

    payload = "{}:{}".format(client_id, client_secret).encode()
    headers = {
        "Authorization": b64encode(payload).decode(),
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    body = {
        "grantType": "clientCredentials",
        "scopes": "read:access",
    }

    content = do_api_post_query(uri="/oauth/token/", body=body, headers=headers)

    print(
        ">>>> Successfully fetched an access token {}****, valid {} seconds.".format(
            content["accessToken"][:5], content["expiresIn"]
        )
    )

    return content["accessToken"]

## N.B. Credentials

Here we call the above functions, and input the file path to our credentials.

N.B. You must have downloaded your client credentials CSV file before proceeding. Please refer to the API documentation if you have not dowloaded them already.

The code then prints the available prices that are callable from the API, and their corresponding Python ticker names are displayed as a list at the bottom of the Output.

In [6]:
# Insert file path to your client credentials here
client_id, client_secret = retrieve_credentials(file_path="/tmp/client_credentials.csv")

# Authenticate:
access_token = get_access_token(client_id, client_secret)

>>>> Found credentials!
>>>> Client_id=875f4****, client_secret=6cdf8****
>>>> Successfully fetched an access token eyJhb****, valid 604799 seconds.


## 2. Latest Slot Release

Here we call the latest slot release and print it in a readable format. This is done using the URL:

__/beta/terminal-slots/releases/latest/__


We then save the entire dataset as a local variable called 'latest'.

In [17]:
# Function to get the latest slot release
def get_latest_slots():
    uri = urljoin(API_BASE_URL,'/beta/terminal-slots/releases/latest/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }
    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
    else:
        print('Bad Request')
    return df

In [18]:
# Call latest slots function
latest = get_latest_slots()
latest.head()

,ReleaseDate,TerminalCode,TerminalName,Total,M+0,M+1,M+2,M+3,M+4,M+5,M+6,M+7,M+8,M+9,M+10,M+11,M+12,M>12
0,2024-11-22,adriatic,Adriatic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2024-11-22,brunsbuttel,Brunsbuttel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2024-11-22,deutsche-ostsee,Deutsche Ostsee,16,1,4,6,5,0,0,0,0,0,0,0,0,0,0
3,2024-11-22,dragon,Dragon,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2024-11-22,dunkerque,Dunkerque,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## 3. Slot Release by Release Date

Here we call the slot data by choosing a specific date and print it in a readable format. This is done using the URL:

__/beta/terminal-slots/releases/{date}/__ where __date__ is the release date, in the "YYYY-MM-DD" string format.


We then save the entire dataset as a local variable called 'release_df'.

In [19]:
# Function to get the slot releases for a specific date
def get_slot_releases(date):
    uri = urljoin(API_BASE_URL, f'/beta/terminal-slots/releases/{date}/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }

    response = requests.get(uri, headers=headers)

    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
        return df

    elif response.content == b'{"errors":[{"code":"object_not_found","detail":"Object not found"}]}':
        print('Bad Date')
        return None
    
    else:
        print('Bad Request')
        return None

In [21]:
# Calling slot release function
release_df = get_slot_releases("2024-10-22")
release_df.head()

,ReleaseDate,TerminalCode,TerminalName,Total,M+0,M+1,M+2,M+3,M+4,M+5,M+6,M+7,M+8,M+9,M+10,M+11,M+12,M>12
0,2024-10-22,adriatic,Adriatic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2024-10-22,brunsbuttel,Brunsbuttel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2024-10-22,deutsche-ostsee,Deutsche Ostsee,20,1,3,4,1,1,1,1,1,1,1,1,1,1,2
3,2024-10-22,dragon,Milford Haven Dragon LNG,3,1,2,0,0,0,0,0,0,0,0,0,0,0,0
4,2024-10-22,dunkerque,Dunkerque,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 4. Terminal List

Here we call the list of terminals and their uuid, and print it in a readable format. This is done using the URL:

__'beta/terminal-slots/terminals/'__


We then save the entire dataset as a local variable called 'terminal_list'.

In [24]:
# Function to get the list of terminals and their uuids (as well as their start and latest release date)
def get_terminal_list():
    uri = urljoin(API_BASE_URL,'beta/terminal-slots/terminals/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }
    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
    else:
        print('Bad Request')
    return df

In [25]:
# Call terminal list function
terminal_list = get_terminal_list()
terminal_list.head()

,uuid,code,name,firstAvailableRelease,latestAvailableRelease
0,00317185-978a-4df5-970c-2c28d3ab893c,grain-lng,Isle of Grain,2023-11-28,2024-11-22
1,0031994e-f370-4927-ba88-a4e7a78c42db,zeebrugge,Zeebrugge,2023-11-28,2024-11-22
2,00338f3f-8875-435d-87a9-f83d9a5c5241,dunkerque,Dunkerque,2023-11-28,2024-11-22
3,00344dd2-5608-4413-a2f4-c52c747a286a,dragon,Dragon,2023-11-28,2024-11-22
4,003497c6-ed32-412f-95ef-c3b1f962464e,brunsbuttel,Brunsbuttel,2023-11-28,2024-11-22


## Fetching Slots Data specific to one terminal

Now that we can see all the terminal data available to us, we can start to define what terminal we want to call slots data for (by referring to 'terminal_list' above).

The first step is to choose which terminal uuid ('my_uuid'), then the request will return all the historical data available for that terminal.

In [35]:
# Function to collect and store historical slots for one specific terminal
def get_individual_terminal(terminal_uuid):
    uri = urljoin(API_BASE_URL, f'/beta/terminal-slots/terminals/{terminal_uuid}/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }
    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
        return df

    elif response.content == b'{"errors":[{"code":"object_not_found","detail":"Object not found"}]}':
        print('Bad Terminal Request')
        return None
    else:
        print('Bad Request')
        return None

In [32]:
# Call individual terminal function (Dragon)
my_uuid = terminal_list[terminal_list['name']=='Dragon']['uuid'].tolist()[0]
dragon_hist_df = get_individual_terminal(my_uuid)
dragon_hist_df

,TerminalCode,TerminalName,ReleaseDate,Total,M+0,M+1,M+2,M+3,M+4,M+5,M+6,M+7,M+8,M+9,M+10,M+11,M+12,M>12
0,dragon,Dragon,2023-11-28,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0
1,dragon,Dragon,2023-11-29,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0
2,dragon,Dragon,2023-11-30,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0
3,dragon,Dragon,2023-12-01,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4,dragon,Dragon,2023-12-04,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,dragon,Dragon,2024-11-18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
247,dragon,Dragon,2024-11-19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
248,dragon,Dragon,2024-11-20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
249,dragon,Dragon,2024-11-21,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


## 5. Historical Slots

Here we collect all the historical slots for each available terminal. This is done using the URL:

__/beta/terminal-slots/terminals/{terminal_uuid}/__

First we define the function that imports the data from the Spark API.

We then call that function, and define the list of terminal uuids.


We save the output as a local variable called 'all_terminals_historical'.

In [33]:
# Function to collect and store each terminal's historical slots data
def get_all_terminal_data(terminal_list):
    terminals_all = pd.DataFrame()
    for i in range(len(terminal_list)):
        print(terminal_list['name'].loc[i])
        terminal_df = get_individual_terminal(terminal_list['uuid'].loc[i])
        time.sleep(0.1)
        terminals_all = pd.concat([terminals_all,terminal_df])
    return terminals_all

In [34]:
# Calling all terminal data function
all_terminal_historical = get_all_terminal_data(terminal_list)
all_terminal_historical.head()

Isle of Grain
Zeebrugge
Dunkerque
Dragon
Brunsbuttel
OLT Toscana
Le Havre
Piombino
EemsEnergyTerminal
Adriatic
Fos Cavaou
Wilhelmshaven 1
South Hook
Inkoo
Deutsche Ostsee
Klaipeda
Gate


,TerminalCode,TerminalName,ReleaseDate,Total,M+0,M+1,M+2,M+3,M+4,M+5,M+6,M+7,M+8,M+9,M+10,M+11,M+12,M>12
0,grain-lng,Isle of Grain,2023-11-28,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,grain-lng,Isle of Grain,2023-11-29,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,grain-lng,Isle of Grain,2023-11-30,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0
3,grain-lng,Isle of Grain,2023-12-01,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4,grain-lng,Isle of Grain,2023-12-04,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
